<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150 align="right">
<img src=images/Acme.jpeg width=100 align="left">

<a id =1></a>
## PKI Secrets Engine


## Business Values:
 * generates dynamic X.509 certificates without going through manual process of generating a private key, CSR, submitting to CA and wait for a verification and signing process to complete

 * relatively short time to live (TTL), often for the applications lifespan.
 * eliminate sharing, unique certificate for each application


### Vault Setup

### Prerequisites
* Running Vault instance (in dev mode) 


In [1]:
vault -version

Vault v1.6.0 (7ce0bd9691998e0443bc77e98b1e2a4ab1e965d4)


In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_TOKEN=s.BCRRJ4g6lqPsYhU7w6aLrTzv
export VAULT_SKIP_VERIFY=true

In [ ]:
vault status

In [ ]:
vault login root

---
### Enable the PKI Secret Engine


In [ ]:
vault secrets enable pki

In [ ]:
vault secrets tune -max-lease-ttl=8760h pki

**Note** the above was the global maximum for this secret engine.  Invidiual roles can restrict the value to be shorter on per-certificate basis.


### Create a certificate
Configure a CA certificate and private key.  Vault can accept an existing key pair or geneate its own self-signed root.

**Note** only one CA certificate is allowed per secrets engine.
Common pattern is to have one mount act as your root CA and to use this CA only to sign intermediate CA CSRs from other PKI secrets engines.

In [ ]:
### Generate root certificate

In [ ]:
vault write pki/root/generate/internal \
   common_name=my-website.com \
   ttl=8760h

**Note** the returned certificate is purely informational.  It and its private key are stored in Vault.
We recommend maintaining your root CA outside of Vault and providing Vault a signed intermediate CA.


### Set URL configuration
Generated certificates can have the CRL (Certifiate Revocation List), a list of digital certificates that have been revoked by the issuing CA, location and the location of the issuing certificate encoded.  These values must be set manually and typically to FQDN associated to the Vault server.  
These can be updated in the future

In [ ]:
vault write pki/config/urls \
   issuing_certificates="http://127.0.0.1:8200/v1/pki/ca" \
   crl_distribution_points="http://127.0.0.1:8200/v1/pki/crl"


### Configure a role 
A role maps a name in Vault to a procedure for generating certificate

In [ ]:
vault write pki/roles/example-dot-com \
   allowed_domains=my-website.com \
   allow_subdomains=true \
   max_ttl=72h

### Generate a new credential

After the secrets engine is configure and a user/machine has a Vault token, it can generae credentials.

In [ ]:
vault write pki/issue/example-dot-com common_name=www.my-website.com

---
### Setting up Intermediate CA
Mount the backend

In [ ]:
vault secrets enable -path=pki_int pki

### Configure an Intermediate CA

In [ ]:
vault secrets tune -max-lease-ttl=43800h pki_int

### Generate intermediate certificate signing request

In [ ]:
vault write pki_int/intermediate/generate/internal common_name="myvault.com Intermediate Authority" \
   ttl=43800h
   

In [ ]:
echo "-----BEGIN CERTIFICATE REQUEST-----
MIICcjCCAVoCAQAwLTErMCkGA1UEAxMibXl2YXVsdC5jb20gSW50ZXJtZWRpYXRl
IEF1dGhvcml0eTCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAL42hybF
uK/RSK2SdhHOwDWxgChJKxnmug+fEBPdrvhg+D3uXWZluXFzAwnnFtgPkK+GmXg0
ZXmTsnx30CtHksADh7GmkrA7PU8f/NdCiPXGrxsibGYD7aNRE8Y7MUIorcYPsY3d
zQ+/a15oM//GQRoMXPL68xnnCnPBP63DC+2YQzr0szQJIuEcoUYFjvI/0f8HMTEr
HfcUP3CMmRxEFLu0UloppK3UGXc9ku7r8BsOzovVOCvC6bzVg+lgBsIIV+2w97IM
VGl26Mu0WxVk+ECkK9Lheb4PX53kZ693+JumpNMQFMLF3FlBqtCO7Cni4HtG/xgo
QcG7dTNeDnLNmJsCAwEAAaAAMA0GCSqGSIb3DQEBCwUAA4IBAQAZcen7BXllqG7O
q87395bLdDA1fPTjb5AROMUuCUsJUZKP9dX4rdPD1q0ASijFLRLWtDNmX1kW9o70
YCX/pF0qNErIu4GejDAHACY4jTQL7hrVIpL5QPPCILF9TSxJLkZgEQfzTnyN0Unn
E1BhKO9NmcNNzE3gRgWlsqOL6Rvd9gr9bAcUQNFWRL13nFFY0JMXocgR6V/t8hUh
gB21kS0X/NqKO8el5DrrTKQjKSsnqQ3JjSLF2YgC1/w6rVjhfD9IIWqaY58KqPZv
BxutThLBiDFdWIpt0SNky3tYxco4dbVwDABr31CoemsE0Iooln4/o5nrHpMMJS0+
hZg20o1D
-----END CERTIFICATE REQUEST-----
" > /tmp/pki_int.csr
cat /tmp/pki_int.csr


### Sign CSR using another certificate authority, 
in this case the root CA generated in the first example

In [ ]:
vault write pki/root/sign-intermediate csr=@/tmp/pki_int.csr format=pem_bundle ttl=32800h

&nbsp;

Now set the intermediate certificate authorities signing certificate to the root-signed certificate

In [ ]:
vault write pki_int/intermediate/set-signed certificate=@signed_certificate.pem

### Set URL configuration
Generate certificates can have the CRL location and the location of the issuing certificate encoded.

In [ ]:
vault write pki_int/config/urls \
  issuing_certificates="http://127.0.0.1:8200/v1/pki_int/ca" \
  crl_distribution_points="http://127.0.0.1:8200/v1/pki_int/crl"

### Configure a role

In [ ]:
vault write pki_int/roles/example-dot-com \
   allowed_domains=example.com \
   allow_subdomain=true max_ttl=72h

### Issue Certificates

In [ ]:
vault write pki_int/issue/example-dot-com common_name=blah.example.com

&nbsp;

Vault now has generated a new set of credentials usign the example-dot-com role coniguration.  Here we see the dynamically generated private key and certificate.  The issuing CA certificate and CA trust chain are returned as well.
The CA Chain returns all the intermeiate authorities in the trust chain.  The root authority is not included since that will usually be trusted by the underlying OS.

---
### Use web UI

Open a browser and visit http://127.0.0.1:8200


&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">